# DB Build
This notebook covers construction and stocking of the Postgres DB used to feed user/sample data to the MetaCapturer

## V1 DB

In [45]:
from sqlalchemy import create_engine, text, inspect

In [93]:
db_username = "pyqt_lims_devs"
db_password = "NOPE"
db_name = "pyqt_lims_database_v1"
db_host = "localhost"  # or the IP address of your Docker host
db_port = "54321"  # the port you mapped in your Docker configuration

connection_string = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

In [94]:
engine = create_engine(connection_string)

In [95]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful!")
        
        # Optionally, check if the database is empty (no tables)
        inspector = inspect(engine)
        if not inspector.get_table_names():
            print("The database is empty (no tables).")
        else:
            print("The database contains tables.")
            
except Exception as e:
    print(f"Error connecting to the database: {e}")



Error connecting to the database: (psycopg2.OperationalError) connection to server at "188.155.85.67", port 54321 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [87]:
### Table drop before re-build
try:
    # Create a MetaData instance
    metadata = MetaData()
    
    # Reflect existing tables
    metadata.reflect(bind=engine)
    
    # Drop all tables
    metadata.drop_all(bind=engine)
    
    print("All tables dropped successfully.")
except:
    print("nothing to drop")


All tables dropped successfully.


In [88]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()

# Define tables
user_tab = Table('user_tab', metadata,
    Column('user_id', Integer, primary_key=True, autoincrement=True),
    Column('user_name', String(100))
)

rg_tab = Table('rg_tab', metadata,
    Column('rg_id', Integer, primary_key=True, autoincrement=True),
    Column('rg_name', String(100))
)

sample_tab = Table('sample_tab', metadata,
    Column('sample_id', Integer, primary_key=True, autoincrement=True),
    Column('sample_name', String(100)),
    Column('sample_type', String(50))
)

instrument_tab = Table('instrument_tab', metadata,
    Column('instrument_id', Integer, primary_key=True, autoincrement=True),
    Column('instrument_name', String(100)),
    Column('observation_type', String(100))
)


project_tab = Table('project_tab', metadata,
    Column('project_id', Integer, primary_key=True, autoincrement=True),
    Column('project_name', String(100)),
    Column('meta_json_filepath', String(200))
)



# Create tables in the database
metadata.create_all(engine)

print("Tables created successfully.")


Tables created successfully.


In [89]:
inspector = inspect(engine)

# Get the list of tables
table_names = inspector.get_table_names()
print("Tables in the database:", table_names)

Tables in the database: ['user_tab', 'rg_tab', 'sample_tab', 'instrument_tab', 'project_tab']


In [90]:
# Insert data into the users table
from sqlalchemy.orm import sessionmaker

# Create a session factory
Session = sessionmaker(bind=engine)

# Insert data into the users table
try:
    # Open a session
    with Session() as session:
        # Insert multiple rows
        session.execute(user_tab.insert(), [
            {'user_name': 'Pete'},
            {'user_name': 'Angus'},
            {'user_name': 'Kostas'}
        ])
        session.execute(rg_tab.insert(), [
            {'rg_name': 'Hani'},
            {'rg_name': 'Algebra'}
        ])
        session.execute(sample_tab.insert(), [
            {'sample_name': 'GLS123456', 'sample_type': 'animal'},
            {'sample_name': 'GLS123457', 'sample_type': 'mineral'}
        ])
        session.execute(project_tab.insert(), [
            {'project_name': 'project_1', 'meta_json_filepath': "/home/pete/Documents/tests_and_vals/roc_pyqt_lims/json_files/project_1.json"},
            {'project_name': 'project_z', 'meta_json_filepath': "/home/pete/Documents/tests_and_vals/roc_pyqt_lims/json_files/project_z.json"}
        ])
        session.execute(instrument_tab.insert(), [
            {'instrument_name': 'an_sem', 'observation_type': "ImageObject"},
            {'instrument_name': 'a_mass_spec', 'observation_type': "Dataset"}
        ])
        # Commit the transaction
        session.commit()
        print("Inserted data successfully.")
except Exception as e:
    print(f"Error occurred: {e}")

# Verify the tables
inspector = inspect(engine)
table_names = inspector.get_table_names()
print("Tables in the database:", table_names)


Inserted data successfully.
Tables in the database: ['user_tab', 'rg_tab', 'sample_tab', 'instrument_tab', 'project_tab']


In [91]:
import pandas as pd

tab_names = ['user_tab', 'rg_tab', 'sample_tab', 'instrument_tab', 'project_tab']

for tab in tab_names:
    with engine.connect() as connection:
        query = text(f"SELECT * FROM {tab};")
        result = connection.execute(query)
        
        # Fetch all results and convert to a DataFrame
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        
    print(df)

   user_id user_name
0        1      Pete
1        2     Angus
2        3    Kostas
   rg_id  rg_name
0      1     Hani
1      2  Algebra
   sample_id sample_name sample_type
0          1   GLS123456      animal
1          2   GLS123457     mineral
   instrument_id instrument_name observation_type
0              1          an_sem      ImageObject
1              2     a_mass_spec          Dataset
   project_id project_name                                 meta_json_filepath
0           1    project_1  /home/pete/Documents/tests_and_vals/roc_pyqt_l...
1           2    project_z  /home/pete/Documents/tests_and_vals/roc_pyqt_l...
